# ETL Pipeline Preparation
Follow the instructions below to help you create your ETL pipeline.
### 1. Import libraries and load datasets.
- Import Python libraries
- Load `messages.csv` into a dataframe and inspect the first few lines.
- Load `categories.csv` into a dataframe and inspect the first few lines.

In [58]:
# import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sqlalchemy import create_engine

%matplotlib inline

In [59]:
# load messages dataset
messages = pd.read_csv('./data/disaster_messages.csv')
messages.head()

,id,message,original,genre
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct


In [60]:
# Print info about dataframe messages
messages.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26248 entries, 0 to 26247
Data columns (total 4 columns):
id          26248 non-null int64
message     26248 non-null object
original    10184 non-null object
genre       26248 non-null object
dtypes: int64(1), object(3)
memory usage: 820.3+ KB


In [61]:
# Print shape of dataframe messages
messages.shape

(26248, 4)

In [62]:
# Print unique values of genre
messages['genre'].unique()

array(['direct', 'social', 'news'], dtype=object)

In [63]:
# Print value counts of each genre
messages['genre'].value_counts()

news      13068
direct    10782
social     2398
Name: genre, dtype: int64

In [64]:
messages['genre'].value_counts().sum()

26248

In [65]:
# Check missing values in dataframe messages
messages.isnull().any()

id          False
message     False
original     True
genre       False
dtype: bool

In [66]:
# Print first 10 observations where original has a null value
messages[messages['original'].isnull()].head(10)

,id,message,original,genre
7433,8365,NOTES: It mark as not enough information,NaN,direct
9902,11186,My thoughts and prayers go out to all the live...,NaN,social
9903,11188,I m sorry for the poor people in Haiti tonight...,NaN,social
9904,11189,RT selenagomez UNICEF has just announced an em...,NaN,social
9905,11192,lilithia yes 5.2 magnitude earthquake hit mani...,NaN,social
9906,11193,RT TheNewsBlotter RT caribnews On Call Interna...,NaN,social
9907,11195,Most Eureka homeowners won&#39 t collect on qu...,NaN,social
9908,11196,Haiti hit by largest earthquake in over 200 ye...,NaN,social
9909,11197,RT pinkelephantpun Earthquake Relief Donate ht...,NaN,social
9910,11198,Praying that today s earthquake in #Haiti mini...,NaN,social


In [67]:
# Print count of observations where original has null value
messages[messages['original'].isnull()].shape[0]

16064

From above, we observe that only column 'original' in dataframe messages have null values and there are 16064 observations with null values in column 'original'.

In [68]:
# load categories dataset
categories = pd.read_csv('./data/disaster_categories.csv')
categories.head()

,id,categories
0,2,related-1;request-0;offer-0;aid_related-0;medi...
1,7,related-1;request-0;offer-0;aid_related-1;medi...
2,8,related-1;request-0;offer-0;aid_related-0;medi...
3,9,related-1;request-1;offer-0;aid_related-1;medi...
4,12,related-1;request-0;offer-0;aid_related-0;medi...


In [69]:
# Print info about dataframe categories
categories.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 26248 entries, 0 to 26247
Data columns (total 2 columns):
id            26248 non-null int64
categories    26248 non-null object
dtypes: int64(1), object(1)
memory usage: 410.2+ KB


In [70]:
# Print shape of dataframe categories
categories.shape

(26248, 2)

In [71]:
# Print unique values in column categories of dataframe categories
categories.loc[0,'categories'].split(';')

['related-1',
 'request-0',
 'offer-0',
 'aid_related-0',
 'medical_help-0',
 'medical_products-0',
 'search_and_rescue-0',
 'security-0',
 'military-0',
 'child_alone-0',
 'water-0',
 'food-0',
 'shelter-0',
 'clothing-0',
 'money-0',
 'missing_people-0',
 'refugees-0',
 'death-0',
 'other_aid-0',
 'infrastructure_related-0',
 'transport-0',
 'buildings-0',
 'electricity-0',
 'tools-0',
 'hospitals-0',
 'shops-0',
 'aid_centers-0',
 'other_infrastructure-0',
 'weather_related-0',
 'floods-0',
 'storm-0',
 'fire-0',
 'earthquake-0',
 'cold-0',
 'other_weather-0',
 'direct_report-0']

In [72]:
# Count of unique categories
len(categories.loc[0,'categories'].split(';'))

36

In [73]:
# Print categories
category_list = []
for category in categories.loc[0,'categories'].split(';'):
    category_list.append(category.split('-')[0])
category_list

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

In [74]:
# defining function cat_count to count the number of categories in each observation of dataframe categories
def cat_count(cat_str):
    '''count the number of categories in each observation of dataframe

    INPUT - 
            cat_str - str, value of column categories in each observation
    OUTPUT -
            number - int, number of categories contained in each observation
    
    '''
    
    cat_list = []
    for cat in cat_str.split(';'):
        cat_list.append(cat.split('-')[0])
    return len(cat_list)

In [75]:
# Find category count of each observation
category_count = categories['categories'].apply(lambda x: cat_count(x))
print(type(category_count),'\n')
print(category_count.head())

<class 'pandas.core.series.Series'> 

0    36
1    36
2    36
3    36
4    36
Name: categories, dtype: int64


In [76]:
# Print unique values in Series category_count
category_count.unique()

array([36], dtype=int64)

In [77]:
# Print value counts of each unique values in Series category_count
category_count.value_counts()

36    26248
Name: categories, dtype: int64

From above, we observe that each observation in dataframe categories have 36 categories mentioned in column 'categories'.

### 2. Merge datasets.
- Merge the messages and categories datasets using the common id
- Assign this combined dataset to `df`, which will be cleaned in the following steps

In [81]:
# merge datasets
df = messages.merge(categories, on='id')
df.head()

,id,message,original,genre,categories
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,related-1;request-0;offer-0;aid_related-0;medi...
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,related-1;request-0;offer-0;aid_related-1;medi...
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,related-1;request-0;offer-0;aid_related-0;medi...
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,related-1;request-1;offer-0;aid_related-1;medi...
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,related-1;request-0;offer-0;aid_related-0;medi...


In [82]:
# print shape of merged dataframe
df.shape

(26386, 5)

In [83]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 26386 entries, 0 to 26385
Data columns (total 5 columns):
id            26386 non-null int64
message       26386 non-null object
original      10246 non-null object
genre         26386 non-null object
categories    26386 non-null object
dtypes: int64(1), object(4)
memory usage: 1.2+ MB


### 3. Split `categories` into separate category columns.
- Split the values in the `categories` column on the `;` character so that each value becomes a separate column. You'll find [this method](https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.Series.str.split.html) very helpful! Make sure to set `expand=True`.
- Use the first row of categories dataframe to create column names for the categories data.
- Rename columns of `categories` with new column names.

In [ ]:
# create a dataframe of the 36 individual category columns
categories = 
categories.head()

In [ ]:
# select the first row of the categories dataframe
row = 

# use this row to extract a list of new column names for categories.
# one way is to apply a lambda function that takes everything 
# up to the second to last character of each string with slicing
category_colnames = 
print(category_colnames)

In [ ]:
# rename the columns of `categories`
categories.columns = category_colnames
categories.head()

### 4. Convert category values to just numbers 0 or 1.
- Iterate through the category columns in df to keep only the last character of each string (the 1 or 0). For example, `related-0` becomes `0`, `related-1` becomes `1`. Convert the string to a numeric value.
- You can perform [normal string actions on Pandas Series](https://pandas.pydata.org/pandas-docs/stable/text.html#indexing-with-str), like indexing, by including `.str` after the Series. You may need to first convert the Series to be of type string, which you can do with `astype(str)`.

In [ ]:
for column in categories:
    # set each value to be the last character of the string
    categories[column] = 
    
    # convert column from string to numeric
    categories[column] = 
categories.head()

### 5. Replace `categories` column in `df` with new category columns.
- Drop the categories column from the df dataframe since it is no longer needed.
- Concatenate df and categories data frames.

In [ ]:
# drop the original categories column from `df`


df.head()

In [ ]:
# concatenate the original dataframe with the new `categories` dataframe
df = 
df.head()

### 6. Remove duplicates.
- Check how many duplicates are in this dataset.
- Drop the duplicates.
- Confirm duplicates were removed.

In [ ]:
# check number of duplicates


In [ ]:
# drop duplicates


In [ ]:
# check number of duplicates


### 7. Save the clean dataset into an sqlite database.
You can do this with pandas [`to_sql` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.to_sql.html) combined with the SQLAlchemy library. Remember to import SQLAlchemy's `create_engine` in the first cell of this notebook to use it below.

In [ ]:
engine = create_engine('sqlite:///InsertDatabaseName.db')
df.to_sql('InsertTableName', engine, index=False)

### 8. Use this notebook to complete `etl_pipeline.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database based on new datasets specified by the user. Alternatively, you can complete `etl_pipeline.py` in the classroom on the `Project Workspace IDE` coming later.